In [1]:
"""
python3 main.py ../../dataset/train.csv ../../dataset/test.csv ../result/cv_results.csv ../result/submission.csv > ../result/logs.txt

make train

"""
import sys
sys.path.append("../fraud_detection/src/")
import time
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from contextlib import contextmanager
import gc 
from util import s_to_time_format, string_to_datetime, hour_to_range, kfold_lightgbm, kfold_xgb
from util import rolling_stats_target_by_cols
#from util import _time_elapsed_between_last_transactions,time_elapsed_between_last_transactions
#from util import num_transaction_in_past_n_days
#from util import add_auto_encoder_feature
#from util import group_target_by_cols_split_by_users
from time import strftime, localtime
import logging
import sys
from config import Configs

# logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))
#log_file = '{}-{}-{}.log'.format(opt.model_name, opt.dataset, strftime("%y%m%d-%H%M", localtime()))
log_file = '../fraud_detection/result/fs_{}.log'.format(strftime("%y%m%d-%H%M", localtime()))
logger.addHandler(logging.FileHandler(log_file))

def group_target_by_cols(df_train, df_test, recipe):
    df = pd.concat([df_train, df_test], axis = 0)
    for m in range(len(recipe)):
        cols = recipe[m][0]
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
            tmp = df[cols + [target]].groupby(cols).agg(method)
            tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
            df_train = df_train.merge(tmp, how='left', on=cols)
            df_test = df_test.merge(tmp, how='left', on=cols)

        # reduced memory    
        del tmp
        gc.collect()
    
    return df_train, df_test

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    logger.info("{} - done in {:.0f}s".format(title, time.time() - t0))
    
def main(args):
    with timer("Process train/test application"):
        #-------------------------
        # load dataset
        #-------------------------
        df_train = pd.read_csv(args.train_file)
        df_test = pd.read_csv(args.test_file)
        #-------------------------
        # pre-processing
        #-------------------------

        for cat in Configs.CATEGORY:
            df_train[cat] = df_train[cat].astype('category') #.cat.codes
            df_test[cat] = df_test[cat].astype('category')
            
        for df in [df_train, df_test]:
            # pre-processing
            df["loctm_"] = df.loctm.astype(int).astype(str)
            df.loctm_ = df.loctm_.apply(s_to_time_format).apply(string_to_datetime)
            # # time-related feature
            df["loctm_hour_of_day"] = df.loctm_.apply(lambda x: x.hour).astype('category')
            df["loctm_minute_of_hour"] = df.loctm_.apply(lambda x: x.minute)
            df["loctm_second_of_min"] = df.loctm_.apply(lambda x: x.second)
            # df["loctm_absolute_time"] = [h*60+m for h,m in zip(df.loctm_hour_of_day,df.loctm_minute_of_hour)]
            df["hour_range"] = df.loctm_.apply(lambda x: hour_to_range(x.hour)).astype("category")
            # removed the columns no need
            df.drop(columns = ["loctm_"], axis = 1, inplace = True)
            # auxiliary fields
            df["day_hr_min"] = ["{}:{}:{}".format(i,j,k) for i,j,k in zip(df.locdt,df.loctm_hour_of_day,df.loctm_minute_of_hour)]
            df["day_hr_min_sec"] = ["{}:{}:{}:{}".format(i,j,k,z) for i,j,k,z in zip(df.locdt,df.loctm_hour_of_day,df.loctm_minute_of_hour,df.loctm_second_of_min)]

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add bacno/cano feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CONAM_AGG_RECIPE_1)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add iterm-related feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.ITERM_AGG_RECIPE)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add conam-related feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CONAM_AGG_RECIPE_2)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add hour-related feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.HOUR_AGG_RECIPE)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add cano/conam feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.CANO_CONAM_COUNT_RECIPE)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add cano/bacno latent feature"):
        df = pd.read_csv("../fraud_detection/features/bacno_latent_features_w_cano.csv")
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")
        df = pd.read_csv("../fraud_detection/features/bacno_cano_latent_features.csv")
        df_train = df_train.merge(df, on = "cano", how = "left")
        df_test = df_test.merge(df, on = "cano", how = "left")

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add locdt-related feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.LOCDT_CONAM_RECIPE)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add mchno-related feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.MCHNO_CONAM_RECIPE)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add scity-related feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.SCITY_CONAM_RECIPE)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add stocn-related feature"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.STOCN_CONAM_RECIPE)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add mchno/bacno latent feature"):
        df = pd.read_csv("../fraud_detection/features/bacno_latent_features_w_mchno.csv")
        df_train = df_train.merge(df, on = "bacno", how = "left")
        df_test = df_test.merge(df, on = "bacno", how = "left")
        df = pd.read_csv("../fraud_detection/features/bacno_mchno_latent_features.csv")
        df_train = df_train.merge(df, on = "mchno", how = "left")
        df_test = df_test.merge(df, on = "mchno", how = "left")

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add time second-level feature on bacno"):
        df_train, df_test = group_target_by_cols(
            df_train, 
            df_test, 
            Configs.HOUR_AGG_SEC_LEVEL_RECIPE_BACNO,
            )
        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add time second-level feature on cano"):
        df_train, df_test = group_target_by_cols(
            df_train, 
            df_test, 
            Configs.HOUR_AGG_SEC_LEVEL_RECIPE_CANO,
            )
        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add time second-level feature on mchno"):
        df_train, df_test = group_target_by_cols(
            df_train, 
            df_test, 
            Configs.HOUR_AGG_SEC_LEVEL_RECIPE_MCHNO,
            )
        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add time second-level feature on csmcu/stocn/scity"):
        df_train, df_test = group_target_by_cols(
            df_train, 
            df_test, 
            Configs.HOUR_AGG_SEC_LEVEL_RECIPE,
            )
        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add time second-level feature on acqic/csmcu/stocn/scity"):
        df_train, df_test = group_target_by_cols(
            df_train, 
            df_test, 
            Configs.HOUR_AGG_SEC_LEVEL_RECIPE_2,
            )
        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add conam-related feature v3"):
        df_train, df_test = group_target_by_cols(
            df_train, 
            df_test, 
            Configs.CONAM_AGG_RECIPE_3,
            )
        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add locdt-related feature v2"):
        df_train, df_test = group_target_by_cols(df_train, df_test, Configs.LOCDT_CONAM_RECIPE_2)

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add conam-related feature v4"):
        df_train, df_test = group_target_by_cols(
            df_train, 
            df_test, 
            Configs.CONAM_AGG_RECIPE_4,
            )
        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))

    with timer("Add cano/mchno latent feature"):
        df = pd.read_csv("../fraud_detection/features/cano_latent_features_w_mchno.csv")
        df_train = df_train.merge(df, on = "cano", how = "left")
        df_test = df_test.merge(df, on = "cano", how = "left")
        df = pd.read_csv("../fraud_detection/features/cano_mchno_latent_features.csv")
        df_train = df_train.merge(df, on = "mchno", how = "left")
        df_test = df_test.merge(df, on = "mchno", how = "left")

        logger.info("Train application df shape: {}".format(df_train.shape))
        logger.info("Test application df shape: {}".format(df_test.shape))   
        
    #return df_train, df_test



    
    with timer("Run LightGBM with kfold"):
#         if args.feature_selection:
#             logger.info("==============Feature Selection==============")
#             for df in [df_train, df_test]:
#                 # drop random features (by null hypothesis)
#                 df.drop(Configs.FEATURE_GRAVEYARD, axis=1, inplace=True, errors='ignore')

#                 # drop unused features features_with_no_imp_at_least_twice
#                 df.drop(Configs.FEATURE_USELESSNESS, axis=1, inplace=True, errors='ignore')

#                 gc.collect()   
#             logger.info("Train application df shape: {}".format(df_train.shape))
#             logger.info("Test application df shape: {}".format(df_test.shape))

        for df in [df_train, df_test]:
            df.drop(columns = ["loctm_hour_of_day",
                               "loctm_minute_of_hour", 
                               "loctm_second_of_min",
                               "day_hr_min",
                               "day_hr_min_sec",
                               ], axis = 1, inplace = True)
   
    return df_train, df_test

#         ITERATION = (5 if args.TEST_NULL_HYPO else 1)
#         feature_importance_df = pd.DataFrame()
#         over_iterations_val_auc = np.zeros(ITERATION)
#         for i in range(ITERATION):
#             logger.info('Iteration %i' %i)
#             if args.model == "lgb":    
#                 iter_feat_imp, over_folds_val_auc = kfold_lightgbm(df_train, df_test, num_folds = args.NUM_FOLDS, args = args, stratified = args.STRATIFIED, seed = args.SEED, logger = logger)
#             elif args.model == "xgb":
#                 iter_feat_imp, over_folds_val_auc = kfold_xgb(df_train, df_test, num_folds = args.NUM_FOLDS, args = args, stratified = args.STRATIFIED, seed = args.SEED, logger = logger)
#             else:
#                 print("Now we only support LightGBM or Xgboost model!")           
#             feature_importance_df = pd.concat([feature_importance_df, iter_feat_imp], axis=0)
#             over_iterations_val_auc[i] = over_folds_val_auc

#         logger.info('============================================\nOver-iterations val f1 score %.6f' %over_iterations_val_auc.mean())
#         logger.info('Standard deviation %.6f\n============================================' %over_iterations_val_auc.std())
    
#     if args.feature_importance_plot == True:
#         from util import display_importances
#         display_importances(feature_importance_df, args.model)
        
#     feature_importance_df_median = feature_importance_df[["feature", "importance"]].groupby("feature").median().sort_values(by="importance", ascending=False)
#     useless_features_df = feature_importance_df_median.loc[feature_importance_df_median['importance'] == 0]
#     feature_importance_df_mean = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)

#     if args.TEST_NULL_HYPO:
#         feature_importance_df_mean.to_csv("../fraud_detection/result/feature_importance-null_hypo.csv", index = True)
#     else:
#         feature_importance_df_mean.to_csv("../fraud_detection/result/feature_importance.csv", index = True)
#         useless_features_list = useless_features_df.index.tolist()
#         logger.info('Useless features: \'' + '\', \''.join(useless_features_list) + '\'')
#     return feature_importance_df_mean

In [2]:
args = {
 "train_file":"/data/yunrui_li/fraud/dataset/train.csv",
 "test_file":"/data/yunrui_li/fraud/dataset/test.csv",
 "result_path":"/data/yunrui_li/fraud/fraud_detection/result/submission.csv",
 "feature_selection":True,
 "feature_importance_plot": False,
 "SEED": 1030,
 "NUM_FOLDS": 5, # 5
 "CPU_USE_RATE":1.0,
 "STRATIFIED": True,
 "NUM_LEAVES":31,
 "COLSAMPLE_BYTREE":1.0,
 "SUBSAMPLE": 1.0,
 "SUBSAMPLE_FREQ": 0,
 "MAX_DEPTH": -1,
 "REG_ALPHA": 0.0,
 "REG_LAMBDA": 0.0,
 "MIN_SPLIT_GAIN": 0.0,
 "MIN_CHILD_WEIGHT": 0.001,
 "MAX_BIN": 255,
 "SCALE_POS_WEIGHT": 3,
 "TEST_NULL_HYPO":False,
 "model": "lgb",
 "ensemble":False,
 "seed":1030,
}

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict(args)
args

{'train_file': '/data/yunrui_li/fraud/dataset/train.csv',
 'test_file': '/data/yunrui_li/fraud/dataset/test.csv',
 'result_path': '/data/yunrui_li/fraud/fraud_detection/result/submission.csv',
 'feature_selection': True,
 'feature_importance_plot': False,
 'SEED': 1030,
 'NUM_FOLDS': 5,
 'CPU_USE_RATE': 1.0,
 'STRATIFIED': True,
 'NUM_LEAVES': 31,
 'COLSAMPLE_BYTREE': 1.0,
 'SUBSAMPLE': 1.0,
 'SUBSAMPLE_FREQ': 0,
 'MAX_DEPTH': -1,
 'REG_ALPHA': 0.0,
 'REG_LAMBDA': 0.0,
 'MIN_SPLIT_GAIN': 0.0,
 'MIN_CHILD_WEIGHT': 0.001,
 'MAX_BIN': 255,
 'SCALE_POS_WEIGHT': 3,
 'TEST_NULL_HYPO': False,
 'model': 'lgb',
 'ensemble': False,
 'seed': 1030}

In [3]:
df_train, df_test = main(args)

Train application df shape: (1521787, 29)
Test application df shape: (421665, 28)
Process train/test application - done in 67s


/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/fraud/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Train application df shape: (1521787, 43)
Test application df shape: (421665, 42)
Add bacno/cano feature - done in 17s
Train application df shape: (1521787, 115)
Test application df shape: (421665, 114)
Add iterm-related feature - done in 107s
Train application df shape: (1521787, 185)
Test application df shape: (421665, 184)
Add conam-related feature - done in 131s
Train application df shape: (1521787, 209)
Test application df shape: (421665, 208)
Add hour-related feature - done in 452s
Train application df shape: (1521787, 210)
Test application df shape: (421665, 209)
Add cano/conam feature - done in 48s
Train application df shape: (1521787, 230)
Test application df shape: (421665, 229)
Add cano/bacno latent feature - done in 5s
Train application df shape: (1521787, 265)
Test application df shape: (421665, 264)
Add locdt-related feature - done in 256s
Train application df shape: (1521787, 272)
Test application df shape: (421665, 271)
Add mchno-related feature - done in 114s
Train app

In [4]:
df = pd.concat([df_train,df_test], axis = 0)

/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/fraud/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
#df.to_csv("../fraud_detection/features/features.csv", index = False)

In [6]:
corr = df.corr()


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

# csmcu & scity positive correlation (幣別與消費城市)
plt.figure(figsize = (14,14))
plt.title('Credit Card Transactions features correlation plot (Pearson)')
sns.heatmap(corr,xticklabels=corr.columns,yticklabels=corr.columns,linewidths=.1,cmap="Reds")
plt.show()

<Figure size 1400x1400 with 2 Axes>

In [1]:
corr

NameError: name 'corr' is not defined

In [7]:
th = 0.990
f = []
feature_seen = []
for feat1 in corr.columns.tolist():
    for feat2, value in corr[feat1].iteritems():
        if feat1!=feat2:
            if value > th or value < (-1)* th:
                if feat2 not in feature_seen:
                    f.append((feat1,feat2,value))
                    feature_seen.append(feat1)

In [16]:
FEATURE_GRAVEYARD = []
for fea_pair in f:
    if fea_pair[0]!=fea_pair[1]:
        if (fea_pair[0]) not in FEATURE_GRAVEYARD and (fea_pair[1]) not in FEATURE_GRAVEYARD:
                FEATURE_GRAVEYARD.append(fea_pair[0])
            
FEATURE_GRAVEYARD = list(set(FEATURE_GRAVEYARD))

In [17]:
len(FEATURE_GRAVEYARD)

163

In [18]:
FEATURE_GRAVEYARD

['sum_conam_BY_bacno_flbmk',
 'max_conam_BY_acqic_day_hr_min_sec',
 'max_iterm_BY_csmcu',
 'var_conam_BY_bacno',
 'count_conam_BY_cano_locdt_scity',
 'mean_conam_BY_bacno_scity_day_hr_min',
 'median_conam_BY_bacno_scity_day_hr_min',
 'min_conam_BY_bacno_scity_day_hr_min',
 'mean_conam_BY_cano_locdt_scity',
 'mean_conam_BY_bacno_stocn_day_hr_min',
 'mean_iterm_BY_stocn',
 'sum_conam_BY_bacno_locdt_mchno',
 'max_conam_BY_bacno_day_hr_min_sec',
 'min_conam_BY_bacno_cano',
 'max_conam_BY_bacno_day_hr_min',
 'max_conam_BY_bacno_locdt_stocn_scity',
 'max_conam_BY_bacno_stocn_day_hr_min_sec',
 'conam',
 'max_conam_BY_bacno',
 'median_conam_BY_cano',
 'count_conam_BY_csmcu',
 'min_conam_BY_bacno_acqic_day_hr_min_sec',
 'count_conam_BY_scity_locdt',
 'mean_conam_BY_bacno_locdt_stocn',
 'min_iterm_BY_bacno_cano',
 'sum_conam_BY_stscd',
 'mean_conam_BY_bacno',
 'count_conam_BY_bacno_cano',
 'count_conam_BY_bacno_stocn_day_hr_min_sec',
 'var_conam_BY_bacno_locdt_mchno',
 'stscd',
 'count_conam_BY_

In [19]:
for pair in f:
    if pair[0]=="max_conam_BY_bacno":
        print (pair)

('max_conam_BY_bacno', 'max_conam_BY_bacno_flbmk', 0.9947683007357537)
